In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, OrderedDict
from simulator import get_simulator
from simulate import simulate
from simulate import mpi_simulate

# Simulating The Chemical Network 

In [ ]:
system = {
    'duration': 999.0, 
    'resolution': 1.0,
    'species': [
        ('i', 0),
        ('I', 0),
        ('a', 0), 
        ('A', 0),
        ('C', 0),
        ('CC', 0),
        ('t', 0), 
        ('T', 0), 
        ('TT', 0),
    ],
    'reactions': [
        ('sigma_i', (), ((1, 'i'), )), 
        ('alpha_i', ((1, 'i'), ), ((1, 'i'), (1, 'I'))), 
        ('F_i', ((1, 'i'), ), ()), 
        ('Gamma', ((1, 'i'), ), ()), 
        ('G_I', ((1, 'I'), ), ()), 
        ('Gamma', ((1, 'I'), ), ()), 

        ('lambda_ta', (), ((1, 't'), (1, 'a'))), 
        ('alpha_a', ((1, 'a'), ), ((1, 'a'), (1, 'A'))), 
        ('alpha_t', ((1, 't'), ), ((1, 't'), (1, 'T'))), 
        ('F_a', ((1, 'a'), ), ()), 
        ('F_t', ((1, 't'), ), ()), 
        ('Gamma', ((1, 'a'), ), ()), 
        ('Gamma', ((1, 't'), ), ()), 
        ('G_A', ((1, 'A'), ), ()), 
        ('G_T', ((1, 'T'), ), ()), 
        ('G_T', ((1, 'TT'), ), ((1, 'T'), )), 
        ('Gamma', ((1, 'A'), ), ()), 
        ('Gamma', ((1, 'T'), ), ()), 

        ('k_C', ((2, 'T'), ), ((1, 'TT'), )), 
        ('k_C', ((1, 'TT'), (1, 'A')), ((1, 'C'), )), 
        ('k_C', ((2, 'C'), ), ((1, 'CC'), )), 
        ('G_C', ((1, 'C'), ), ((1, 'TT'), )), 
        ('G_C', ((1, 'CC'), ), ((1, 'TT'), (1, 'C'))), 
        ('Gamma', ((1, 'TT'), ), ()), 
        ('Gamma', ((1, 'C'), ), ()), 
        ('Gamma', ((1, 'CC'), ), ()), 
        
    ], 
    'variables': [
        ('sigma_i', 20.0), 
        ('alpha_i', 200.0), 
        ('mu_i', 5.0), 
        ('sigma', 20.0), 
        ('k_0', 0.05), 
        ('alpha_t', 200.0), 
        ('nu', 1.6), 
        ('k_C', 1000.0), 
        ('mu_ta', 5.0), 
        ('f_T', 20.0), # change to beta_T
        ('omega', 0.2), 
        ('delta', 1.5),
        ('mu', 100.0), 
        #('mu', 50.0), 
        ('epsilon', 0.01), 
        ('kappa_M', 1.0),
        ('kappa_I', 1.0), 
        ('eta_p', 0.5), 
        ('kappa_A', 1.0), 
        ('kappa_C', 0.1), 
        ('kappa_T', 0.0), 
        #('kappa_T1', 0.0), 
        #('kappa_T2', 0.0), 
        ('eta_t', 0.5), 
        ('gamma', 1.0), 
        ('phi', 1.0), 
        ('b_c', 0.25), # change to beta_g
    ], 
    'functions': [
        ('alpha_a', 'alpha_t * nu'), 
        ('T_free', 'T + TT'), 
        ('lambda_ta', 'sigma / (1 + k_0 * (A + C + CC))'), 
        ('F_t', 'mu_ta + f_T * (T + TT)'), # change to beta_t 
        ('F_a', 'mu_ta + f_T * (T + TT) * omega'), # change to beta_a
        ('F_i', 'mu_i  + f_T * (T + TT) * delta'),
        ('G1', 'mu / (epsilon + (kappa_I_c * kappa_I) * I + (kappa_M_c * kappa_M) * (kappa_A * A + kappa_C * (C + CC) + (kappa_T1 * kappa_T) * (T + TT)))'),
        ('G2', 'mu / (epsilon + (kappa_I_c * kappa_I) * I + (kappa_M_c * kappa_M) * (kappa_A * A + kappa_C * (C + CC) + (kappa_T2 * kappa_T) * (T + TT)))'),
        ('G_I', 'kappa_I_c * kappa_I * (G1 + G2)'),
        ('G_A', 'kappa_M_c * kappa_M * kappa_A * (G1 + G2)'),
        ('G_C', 'kappa_M_c * kappa_M * kappa_C * (G1 + G2)'),
        ('G_T', 'kappa_M_c * kappa_M * kappa_T * (kappa_T1 * G1 + kappa_T2 * G2)'),
        ('Gamma', '(gamma * phi) / (1.0 + b_c * phi * (T + TT))'),
        ('kappa_M_c', 'eta_p'),
        ('kappa_I_c', '1.0 - eta_p'),
        ('kappa_T1', 'eta_t'),
        ('kappa_T2', '1.0 - eta_t'),
    ], 
    'targets': [
        'time', 
        'i', 
        'I',
        'A',
        'C',
        'CC',
        'T', 
        'TT',
        'T_free',
    ],
}

In [ ]:
df = pd.DataFrame(OrderedDict((t, d) for t, d in zip(system['targets'], get_simulator(system)(0))))
df.plot(x='time', y=['T_free', 'i', 'I'], kind='line', figsize=(12, 8))

# Measurements Of Toxicity Over Parameter Space 

In [ ]:
from event_detection import event_detection
from pscan import pscan_view

processing = (
    (event_detection, ('time', 'T_free', 'I')), 
)

axes = (
    #('sigma_i', np.arange(0.0, 20.0 + 1.0, 1.0)),
    
    #('mu', (50, 100)),

    #('kappa_I', (0, 1)),

    #('kappa_T', (0, 0.0000001, 0.000001, 0.00001)),
    
    #('eta_p', (0, 0.5)),
    ('eta_p', np.arange(0.0, 1.0 + 0.1, 0.2)),

    #('eta_t', (0, 0.5, 1)),
    #('eta_t', np.arange(0.0, 1.0 + 0.1, 0.1)),
)

#pspace, data = simulate(system, processing=processing, batchsize=10, axes=axes)
pspace, pscans = mpi_simulate(system, processing=processing, batchsize=10, axes=axes)

# Viewing Measurements 

In [ ]:
pscan = pscans[0]
subspace = {
    #'kappa_T': 0.0000001, 
    #'kappa_T': 0, 
    #'kappa_I': 0, 
    #'eta_t': 0.5, 
}
targets = ('time', 'T_free', 'I')

selection = pscan_view(pscan, subspace, targets)
selection.shape, selection.columns

In [ ]:
trajectory = 0
target = 'T_free'

time_selection = selection.loc[selection['Target'] == 'time'].reset_index()
target_selection = selection.loc[selection['Target'] == target].reset_index()

n_fig = target_selection.shape[0]
f, axes = plt.subplots(n_fig, 1, figsize=(12, 2 * n_fig))
f.subplots_adjust(wspace=0.8, hspace=0.8)
for (t, time), (l, location), ax in zip(time_selection.iterrows(), target_selection.iterrows(), axes):
    subspace = location[pspace.columns]
    subspace = ', '.join(('%s: %s' % (i, j)) for i, j in zip(subspace.index, subspace.values))
    ax.set_title('location: %s' % subspace)
    counts = location['Batch'][trajectory, :]
    time = time['Batch'][trajectory, :]
    ax.plot(time, counts, linewidth=3)
    events = location['Events']
    if events and sum([len(e) for e in events]):
        parameters = location['Parameters']
        high, low = parameters['high'], parameters['low']
        medium = (high + low) / 2.0
        ax.plot([time[0] - 1, time[-1] + 1], [low, low], color='k', ls='--')
        ax.plot([time[0] - 1, time[-1] + 1], [medium, medium], color='k', ls='--')
        ax.plot([time[0] - 1, time[-1] + 1], [high, high], color='k', ls='--')
        for i, j in events[trajectory]:
            ax.plot([time[i], time[j]], [medium, medium], color='r', lw=5, marker='s')

In [ ]:
x = 'eta_p'
ys = [['Toxic Probability'], ['Mean Event Duration']]

print(target_selection[x])

n = len(ys)
f, axes = plt.subplots(n, 1, figsize=(12, n * 6))
for ax, targets in zip(axes, ys):
    target_selection.plot(x=x, y=targets, ax=ax)
    ax.set_title('%s Versus %s' % (', '.join(targets), x))
    ax.set_xlabel(x)
    ax.set_ylabel(', '.join(targets))